In [53]:
import pandas as pd
import sqlite3

In [55]:
launches = pd.read_csv('launch_df.csv')
launches.head()

,launch_id,position,status,launch_place,launch_data
0,1,1 E,retired,2700,1999-02-26
1,2,2 E,retired,2790,1993-05-12
2,3,2 E,active,3669,2017-06-23
3,4,3 E,active,6396,2022-09-07
4,5,3 E,deorbited,2283,1995-12-06


In [57]:
objects = pd.read_csv('objects_df.csv')
objects.head()

,launch_id,name,model,operator,object_id
0,1,BADR 3 (Arabsat 3A),Spacebus-3000B2,Arabsat,1
1,2,Astra 1C,HS-601,SES S.A.,2
2,3,BulgariaSat-1,LS-1300,BulgariaSat,3
3,4,Eutelsat Konnect VHTS,Spacebus NEO,Eutelsat Communications S.A.,4
4,5,Telecom 2C,Eurostar-2000,France Telecom,5


In [59]:

conn = sqlite3.connect('space_objects.db')


launches.to_sql('launches', conn, if_exists='replace', index=False,
                  dtype={
                      'launch_id': 'INTEGER PRIMARY KEY',
                      'position': 'TEXT',
                      'status': 'TEXT', 
                      'launch_place': 'TEXT',
                      'launch_data': 'DATA'
                  })

objects.to_sql('objects', conn, if_exists='replace', index=False,
                 dtype={
                     'object_id': 'INTEGER PRIMARY KEY',
                     'launch_id': 'INTEGER',
                     'name': 'TEXT',
                     'model': 'TEXT',
                     'operator': 'TEXT',
                     'FOREIGN KEY (launch_id)': 'REFERENCES launches(launch_id)'
                 })

conn.close()

In [61]:
conn = sqlite3.connect('space_objects.db')

In [63]:
def query(conn, query):
    print(pd.read_sql(query, conn))

In [65]:
query(conn, 'SELECT * FROM launches')

     launch_id position     status launch_place launch_data
0            1      1 E    retired         2700  1999-02-26
1            2      2 E    retired         2790  1993-05-12
2            3      2 E     active         3669  2017-06-23
3            4      3 E     active         6396  2022-09-07
4            5      3 E  deorbited         2283  1995-12-06
..         ...      ...        ...          ...         ...
622        623      1 W     active         3049  2009-10-29
623        624      1 W     active         4600  2015-04-26
624        625      1 W     active         1500  1999-02-26
625        626      1 W     active         5576  2004-06-16
626        627      1 W     active         1960  2008-02-11

[627 rows x 5 columns]


In [67]:
query(conn, "SELECT launch_id, position, status FROM launches where status = 'retired'")

     launch_id position   status
0            1      1 E  retired
1            2      2 E  retired
2            9      4 E  retired
3           10      4 E  retired
4           18     10 E  retired
..         ...      ...      ...
136        605      8 W  retired
137        606      8 W  retired
138        612      7 W  retired
139        616      4 W  retired
140        617      4 W  retired

[141 rows x 3 columns]


In [69]:
query(conn, "SELECT * from launches where launch_data between '1999-02-26' and '2022-09-07'")

     launch_id position     status launch_place launch_data
0            1      1 E    retired         2700  1999-02-26
1            3      2 E     active         3669  2017-06-23
2            4      3 E     active         6396  2022-09-07
3            6      3 E  deorbited         3160  2007-12-21
4            7      3 E     active         3050  2010-08-04
..         ...      ...        ...          ...         ...
466        623      1 W     active         3049  2009-10-29
467        624      1 W     active         4600  2015-04-26
468        625      1 W     active         1500  1999-02-26
469        626      1 W     active         5576  2004-06-16
470        627      1 W     active         1960  2008-02-11

[471 rows x 5 columns]


In [73]:
query(conn, "select distinct status from launches")

      status
0    retired
1     active
2  deorbited
3     failed
4   disabled
5    testing
6     moving
7    planned


In [79]:
query(conn, "Select status, count(*) as quntity from launches group by status having count(*) > 5 and count(*) < 100;")  

      status  quntity
0  deorbited       72
1   disabled       10
2     failed       28
3     moving        6
4    testing        6


In [81]:
query(conn, """select l.launch_id, l.status, o.name, o.model
                from launches l
                join objects o on o.launch_id = l.launch_id
""")

     launch_id     status                                               name  \
0            1    retired                                BADR 3 (Arabsat 3A)   
1            2    retired                                           Astra 1C   
2            3     active                                      BulgariaSat-1   
3            4     active                              Eutelsat Konnect VHTS   
4            5  deorbited                                         Telecom 2C   
..         ...        ...                                                ...   
622        623     active                        Thor 6 (Intelsat 1W, IS-1W)   
623        624     active                                             Thor 7   
624        625     active                                          Skynet 4E   
625        626     active  Intelsat 10-02 + MEV2 (Intelsat 1002, Intelsat...   
626        627     active                                   Thor 5 (Thor 2R)   

               model  
0    Spacebus-30

In [83]:
query(conn, "SELECt distinct operator from objects")

                                     operator
0                                     Arabsat
1                                    SES S.A.
2                                 BulgariaSat
3                Eutelsat Communications S.A.
4                              France Telecom
..                                        ...
81                  AOneSat Communications AG
82          Agence Spatiale Algerienne (ASAL)
83       Nilesat (The Egyptian Satellite Co.)
84  JAXA (Japan Aerospace Exploration Agency)
85                       Government of Israel

[86 rows x 1 columns]


In [89]:
query(conn, """
    select l.launch_id, l.status, o.operator
    from launches l
    join objects o on l.launch_id = o.launch_id
    where o.operator = 'Arabsat';
""")

   launch_id   status operator
0          1  retired  Arabsat
1         39   active  Arabsat
2         53   active  Arabsat
3         54   active  Arabsat
4         55   active  Arabsat
5         56  retired  Arabsat
6         57   active  Arabsat
7         64   active  Arabsat
8         65   active  Arabsat
9         77  retired  Arabsat


In [91]:
query(conn, """
    select *
    from launches
    where status = 'retired' and position = '2 E';
""")

   launch_id position   status launch_place launch_data
0          2      2 E  retired         2790  1993-05-12


In [97]:
query(conn, """
    select * , 
        rank(*) over(partition by status order by launch_id) as order_number
    from launches;
""")

     launch_id position   status launch_place launch_data  order_number
0            3      2 E   active         3669  2017-06-23             1
1            4      3 E   active         6396  2022-09-07             2
2            7      3 E   active         3050  2010-08-04             3
3            8      3 E   active         5967  2014-05-27             4
4           11      5 E   active         4600  2007-11-18             5
..         ...      ...      ...          ...         ...           ...
622        159     68 E  testing         4700  2024-11-18             2
623        261    113 E  testing         4000  2024-02-20             3
624        267    116 E  testing         3500  2024-11-11             4
625        474     89 W  testing         6400  2023-05-01             5
626        594     16 W  testing         4100  2006-03-11             6

[627 rows x 6 columns]


In [101]:
query(
    conn,
    """
    SELECT *
    FROM launches
    where launch_place > 4000 and launch_place < 6000
    order by launch_data;
    """    
)

     launch_id position     status launch_place launch_data
0          355    178 E    retired         4215  1989-10-27
1          503     80 W  deorbited         4215  1990-03-14
2          103     47 E    retired         4330  1991-10-29
3          331    157 E  deorbited         4180  1995-05-17
4          553     53 W    retired         4180  1996-03-14
..         ...      ...        ...          ...         ...
216         94     42 E     active         4300  2024-07-08
217        159     68 E    testing         4700  2024-11-18
218         97     44 E    testing         5000  2025-01-04
219        251    110 E     active         5500  2025-02-22
220        213     92 E     active         5500  2025-06-20

[221 rows x 5 columns]


In [103]:
query(
    conn,
    """
    SELECT l.launch_id, o.model, count(*) as quantity
    from objects o
    join launches l on l.launch_id = o.launch_id
    group by model
    having count(*) > 5
    """
)


     launch_id                                               name  \
0            1                                BADR 3 (Arabsat 3A)   
1            2                                           Astra 1C   
2            3                                      BulgariaSat-1   
3            4                              Eutelsat Konnect VHTS   
4            5                                         Telecom 2C   
..         ...                                                ...   
622        623                        Thor 6 (Intelsat 1W, IS-1W)   
623        624                                             Thor 7   
624        625                                          Skynet 4E   
625        626  Intelsat 10-02 + MEV2 (Intelsat 1002, Intelsat...   
626        627                                   Thor 5 (Thor 2R)   

               model                                           operator  \
0    Spacebus-3000B2                                            Arabsat   
1             HS-601 

In [ ]:
conn.close()